In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import re
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [2]:
games = pd.read_csv('./Resources/games.csv')
pff_scouting_data = pd.read_csv('./Resources/pffScoutingData.csv')
players = pd.read_csv('./Resources/players.csv')
plays = pd.read_csv('./Resources/plays.csv')

week_1 = pd.read_csv('./Resources/week1.csv')
week_2 = pd.read_csv('./Resources/week2.csv')
week_3 = pd.read_csv('./Resources/week3.csv')
week_4 = pd.read_csv('./Resources/week4.csv')
week_5 = pd.read_csv('./Resources/week5.csv')
week_6 = pd.read_csv('./Resources/week6.csv')
week_7 = pd.read_csv('./Resources/week7.csv')
week_8 = pd.read_csv('./Resources/week8.csv')

In [3]:
tracking = week_1.append([
    week_2,
    week_3,
    week_4,
    week_5,
    week_6,
    week_7,
    week_8,
])

In [4]:
# Filter for blocking linemen
blocker_pff_scouting_data = pff_scouting_data[pff_scouting_data['pff_nflIdBlockedPlayer'].notna()]
blocker_pff_scouting_data

,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
4,2021090900,97,40151,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0
7,2021090900,97,42377,Pass Block,LT,NaN,NaN,NaN,0.0,0.0,1.0,0.0,42403.0,PP,0.0
9,2021090900,97,42404,Pass Block,LG,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,PP,0.0
16,2021090900,97,46163,Pass Block,RG,NaN,NaN,NaN,0.0,0.0,1.0,0.0,53441.0,SW,0.0
18,2021090900,97,52421,Pass Block,RT,NaN,NaN,NaN,0.0,0.0,1.0,0.0,53441.0,PP,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188238,2021110100,4433,43425,Pass Route,HB-R,NaN,NaN,NaN,0.0,0.0,0.0,0.0,43326.0,CH,1.0
188239,2021110100,4433,43695,Pass Block,LG,NaN,NaN,NaN,0.0,0.0,0.0,1.0,43338.0,SW,0.0
188240,2021110100,4433,44835,Pass Route,SLiWR,NaN,NaN,NaN,0.0,0.0,0.0,0.0,42406.0,CH,0.0
188242,2021110100,4433,46103,Pass Block,RG,NaN,NaN,NaN,0.0,0.0,0.0,0.0,43326.0,PP,0.0


In [5]:
tracking_with_blocker_id = tracking.merge(
    blocker_pff_scouting_data[['gameId', 'playId', 'nflId', 'pff_nflIdBlockedPlayer', 'pff_sackAllowed']],
    on=['gameId', 'playId', 'nflId'], how='inner')

block_tracking = tracking_with_blocker_id.merge(
    tracking,
    left_on=['gameId', 'playId', 'frameId', 'pff_nflIdBlockedPlayer'],
    right_on=['gameId', 'playId', 'frameId', 'nflId'],
    suffixes=('Blocker', 'Blocked'),
    how='left')

block_tracking

,gameId,playId,nflIdBlocker,frameId,timeBlocker,jerseyNumberBlocker,teamBlocker,playDirectionBlocker,xBlocker,yBlocker,...,teamBlocked,playDirectionBlocked,xBlocked,yBlocked,sBlocked,aBlocked,disBlocked,oBlocked,dirBlocked,eventBlocked
0,2021090900,97,40151.0,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,...,DAL,right,43.47,25.14,0.00,0.00,0.01,43.02,104.69,None
1,2021090900,97,40151.0,2,2021-09-10T00:26:31.200,66.0,TB,right,42.16,23.97,...,DAL,right,43.46,25.14,0.00,0.00,0.00,39.18,102.97,None
2,2021090900,97,40151.0,3,2021-09-10T00:26:31.300,66.0,TB,right,42.15,23.97,...,DAL,right,43.46,25.15,0.00,0.01,0.01,231.69,65.37,None
3,2021090900,97,40151.0,4,2021-09-10T00:26:31.400,66.0,TB,right,42.15,23.98,...,DAL,right,43.44,25.14,0.04,0.49,0.01,311.12,274.26,None
4,2021090900,97,40151.0,5,2021-09-10T00:26:31.500,66.0,TB,right,42.14,23.99,...,DAL,right,43.41,25.16,0.21,1.38,0.03,278.27,285.44,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982415,2021110100,4433,52507.0,54,2021-11-02T03:20:26.500,74.0,NYG,right,21.51,26.75,...,KC,right,21.02,24.96,0.80,0.87,0.08,58.08,254.58,None
1982416,2021110100,4433,52507.0,55,2021-11-02T03:20:26.600,74.0,NYG,right,21.63,26.97,...,KC,right,20.94,24.95,0.86,1.05,0.08,51.95,263.47,None
1982417,2021110100,4433,52507.0,56,2021-11-02T03:20:26.700,74.0,NYG,right,21.75,27.19,...,KC,right,20.85,24.95,0.89,0.83,0.09,46.71,270.02,None
1982418,2021110100,4433,52507.0,57,2021-11-02T03:20:26.800,74.0,NYG,right,21.88,27.45,...,KC,right,20.76,24.96,0.94,0.79,0.09,41.73,278.43,None


In [6]:
def generate_plot(tracking_slice):
    gameId_vis = tracking_slice['gameId'].iloc[0]
    playId_vis = tracking_slice['playId'].iloc[0]
    nflIdBlocker_vis = tracking_slice['nflIdBlocker'].iloc[0]

    vis_data = block_tracking[
        (block_tracking['gameId'] == gameId_vis) &
        (block_tracking['playId'] == playId_vis) &
        (block_tracking['nflIdBlocker'] == nflIdBlocker_vis)
    ]

    blocker_coords = vis_data[['xBlocker', 'yBlocker', 'frameId']]
    blocker_coords = blocker_coords.rename(columns={'xBlocker': 'x', 'yBlocker': 'y'})
    blocker_coords['team'] = "Offense"

    blocked_coords = vis_data[['xBlocked', 'yBlocked', 'frameId']]
    blocked_coords = blocked_coords.rename(columns={'xBlocked': 'x', 'yBlocked': 'y'})
    blocked_coords['team'] = "Defense"

    formation_data = block_tracking[
        (block_tracking['gameId'] == gameId_vis) &
        (block_tracking['playId'] == playId_vis) &
        (block_tracking['frameId'] == 1)
    ]

    blocker_form = formation_data[['xBlocker', 'yBlocker']]
    blocker_form = blocker_form.rename(columns={'xBlocker': 'x', 'yBlocker': 'y'})
    blocker_form['team'] = "Offense"
    blocker_form['frameId'] = 20

    blocked_form = formation_data[['xBlocked', 'yBlocked']]
    blocked_form = blocked_form.rename(columns={'xBlocked': 'x', 'yBlocked': 'y'})
    blocked_form['team'] = "Defense"
    blocked_form['frameId'] = 20

    block_coords = blocker_coords.append(blocked_coords, ignore_index=True)
    block_coords = block_coords.append(blocker_form, ignore_index=True)
    block_coords = block_coords.append(blocked_form, ignore_index=True)

    fig = px.scatter(block_coords, x="y", y="x", color="team", size='frameId')
    fig.show()

In [8]:
tracking_slice = block_tracking.sample(n=1, frac=None, replace=False)
generate_plot(tracking_slice)